In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob

In [107]:
'''
Fields
market_cap_usd
name
price_usd
symbol
'''
exclusion_list = []# ['BCC','XRP','USDT']
capital = 20000

response = requests.get('https://api.coinmarketcap.com/v1/ticker/')
json_data = json.loads(response.text)
i = 0
crypto_df = {}
for cc in json_data:
    temp_df = pd.DataFrame(cc, index = [i])[['symbol','name','market_cap_usd','price_usd']]
    if len(crypto_df) == 0:
        crypto_df = temp_df
    else:
        crypto_df = pd.concat([crypto_df,temp_df],axis = 0)
    i += 1
    
crypto_df['market_cap_usd'] = pd.to_numeric(crypto_df['market_cap_usd'])
crypto_df['price_usd'] = pd.to_numeric(crypto_df['price_usd'])

crypto_df = crypto_df.sort_values(by = 'market_cap_usd', ascending = False)
crypto_df = crypto_df[crypto_df['symbol'].isin(exclusion_list) == False]#[:30]

crypto_df['weight'] = crypto_df['market_cap_usd']/crypto_df['market_cap_usd'].sum()
crypto_df['dollar_weight'] = crypto_df['weight']*capital
crypto_df['expected_holdings'] = crypto_df['dollar_weight']/crypto_df['price_usd']

crypto_list = [(lambda x: x.lower().replace(" ", "-"))(x) for x in list(crypto_df['name'])]

In [150]:
# Starting table: <table class="table">
# Ending table: </table>

# site: https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=20171111

ticker = 'bitcoin'
start_date = '20130428'
end_date = '20171111'

def coinmktcapPull(ticker, start_date, end_date):
    
    wrongticker = "Sorry, we couldn't find your page"

    date_range = '/historical-data/?start=' + start_date + '&end=' + end_date
    site = 'https://coinmarketcap.com/currencies/' + ticker + date_range
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    #soup = soup.prettify()

    if wrongticker in soup:
        return ticker, ticker

    table_start = '<table class="table">'
    table_end = '</table>'
    if len(soup.find_all('table')) == 0:
        return ticker, ticker

    table = soup.find_all('table')[0]    

    # Parsing table from beautiful soup text
    row_marker = 0
    dates = []
    opens = []
    highs = []
    lows = []
    closes = []
    volumes = []
    mktcaps = []
    for row in table.find_all('tr'):
        column_marker = 0
        columns = str(row).split('\n')
        if 'Date' in columns[1]:
            continue
        else:
            dates = dates + [columns[1].split('<td class="text-left">')[1].split('</td>')[0]]
            opens = opens + [columns[2].split('<td>')[1].split('</td>')[0]]
            highs = highs + [columns[3].split('<td>')[1].split('</td>')[0]]
            lows = lows + [columns[4].split('<td>')[1].split('</td>')[0]]
            closes = closes + [columns[5].split('<td>')[1].split('</td>')[0]]
            volumes = volumes + [columns[6].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
            mktcaps = mktcaps + [columns[7].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
    '''
    crypto_dict = {ticker + '_open': opens,
                   ticker + '_high': highs,
                   ticker + '_low': lows,
                   ticker + '_close': closes,
                   ticker + '_volume': volumes,
                   ticker + '_mktcap': mktcaps}
    '''
    crypto_closes = {ticker + '_close': closes}
    crypto_mktcaps = {ticker + '_mktcap': mktcaps}
    close_df = pd.DataFrame(crypto_closes, index = dates)
    mktcap_df = pd.DataFrame(crypto_mktcaps, index = dates)
    
    #coinmktcapdf[ticker + '_open'] = pd.to_numeric(coinmktcapdf[ticker + '_open'])
    #coinmktcapdf[ticker + '_high'] = pd.to_numeric(coinmktcapdf[ticker + '_high'])
    #coinmktcapdf[ticker + '_low'] = pd.to_numeric(coinmktcapdf[ticker + '_low'])
    close_df[ticker + '_close'] = pd.to_numeric(close_df[ticker + '_close'])
    #coinmktcapdf[ticker + '_volume'] = pd.to_numeric(coinmktcapdf[ticker + '_volume'])
    mktcap_df[ticker + '_mktcap'] = pd.to_numeric(mktcap_df[ticker + '_mktcap'])
    close_df.index = pd.to_datetime(close_df.index)
    mktcap_df.index = pd.to_datetime(mktcap_df.index)
    return close_df, mktcap_df

In [151]:
all_close = []
all_mktcap = []
wrong_names = []

start_date = '20130428'
end_date = '20171111'
for tick in crypto_list:
    temp_close, temp_mktcap = coinmktcapPull(tick, start_date, end_date)
    if type(temp_df) == str:
        wrong_names += [temp_close]
    else:
        all_close += [temp_close]
        all_mktcap += [temp_mktcap]
        
correct_names = ['stellar', 'bytecoin-bcn','metaverse','byteball','attention-token-of-media','rlc']

for tick in correct_names:
    temp_close, temp_mktcap = coinmktcapPull(tick, start_date, end_date)
    if type(temp_df) == str:
        wrong_names += [temp_close]
    else:
        all_close += [temp_close]
        all_mktcap += [temp_mktcap]

In [171]:
cc_close = pd.concat(filter(lambda x: type(x) != str, all_close), axis = 1).fillna(0)
cc_mcaps = pd.concat(filter(lambda x: type(x) != str, all_mktcap), axis = 1).fillna(0)

cc_index = cc_close[['bitcoin_close']]
cc_index.columns = ['index']

In [168]:
# cc_close.iloc[0].sort_values(ascending = False)

In [ ]:
for i in range(len(cc_index)):
    cc_index.iloc[i] = 0

C:\Users\Fang\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
cc_index